# Bootstrap Calibrations

In [1]:
# imports
from desispec import bootcalib as desiboot
import numpy as np
from astropy.modeling import models, fitting
from xastropy.xutils import afits as xafits

## Find peaks

In [2]:
# Read flat
flat = fits.open('/Users/xavier/DESI/Wavelengths/pix-b0-00000001.fits')[0].data

In [3]:
reload(desiboot)
xpk, ypos = desiboot.find_fiber_peaks(flat)

Found 500 fibers
Found 20 bundles


## Trace the flat spectra

### Crude

In [4]:
reload(desiboot)
xset, xerr = desiboot.trace_crude_init(flat,xpk[0:50],ypos)

### Polynomial fits (test)

In [ ]:
yval = np.arange(4096)
ii=4
xval = xset[:,ii]
gdval = xerr[:,ii] < 999.
dfit0 = xafits.func_fit(yval[gdval],xval[gdval],'legendre',6)

In [ ]:
fitv = xafits.func_val(yval,dfit0)

In [ ]:
xdb.xplot(yval,fitv, xtwo=yval,ytwo=xset[:,ii],mtwo='+')

In [ ]:
xdb.xplot(yval[gdval],fitv[gdval]-xval[gdval], scatter=True)

### Polynomial fits (True)

In [5]:
reload(desiboot)
xfit, fits = desiboot.fit_traces(xset,xerr)#[:,0:5],xerr[:,0:5])

### QA

In [ ]:
reload(desiboot)
desiboot.fiber_trace_qa(flat,xfit)

## PSF

### Sigma for each fiber (initial guess)

In [ ]:
reload(desiboot)
flux = desiboot.fiber_gauss(flat,xfit,xerr)

In [ ]:
g_init = models.Gaussian1D(amplitude=1., mean=0., stddev=1.)

In [ ]:
g_init.amplitude.fixed = True
g_init.amplitude.value=2.
g_init.mean.fixed = True

In [ ]:
print(g_init.amplitude)

In [ ]:
reload(xafits)
gdval = xerr[:,0] < 900.
yval = np.arange(4096)
dfitf,mask = xafits.iter_fit(yval[gdval],flux[gdval],'legendre',17, max_rej=10)
#
fitv = xafits.func_val(yval,dfitf)
xdb.xplot(yval,fitv, xtwo=yval,ytwo=flux,mtwo='+')